In [24]:
import requests 

from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

from time import sleep
import pandas as pd
import numpy as np
import datetime
import os

# Inicializar parametros de entrada

In [3]:
# PONER USER-AGENT PROPIO:
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/118.0.0.0 Safari/537.36"

# Options:
opts = Options()
opts.add_argument("--disable-extensions")
opts.add_argument(f"User-Agent={user_agent}")

# url_infoempleo:
url_infoempleo = "https://www.infoempleo.com/trabajo/area-de-empresa_tecnologia-e-informatica/"

# Saco las URL de cada puesto en cada página:

In [17]:
def url_scraper(url, nuevas= True):
    
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()
    
    browser.get(url)
    sleep(2)
    
    # Acepto cookies
    browser.find_element(By.ID, "onetrust-accept-btn-handler").click()
    sleep(1)
    
    url_empleos = []
    while True:

        try:
            browser.find_element(By.ID, "lightbox").find_element(By.CLASS_NAME, "close").click()
            sleep(1)
        except:
            pass
        
        if nuevas:
            # Boton ofertas en las últimas 24h:
            browser.find_element(By.ID, "fechapublicacion1").click()
            sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")
        empleos = soup.find("div", class_= "main-content").find_all("li", class_= "offerblock")

        for url in empleos:
            try:
                url_oferta = "https://www.infoempleo.com" + url.find("a")["href"]
                url_empleos.append(url_oferta)
            except:
                pass

        try:
            # Hago scroll hasta el final:
            elemento_objetivo = browser.find_elements(By.CLASS_NAME, "related-offer-item")[-1]
            browser.execute_script("arguments[0].scrollIntoView(true);", elemento_objetivo)
            sleep(1)
            # Siguiente página:
            browser.find_element(By.CLASS_NAME, "pagination").find_element(By.CLASS_NAME, "next").click()
            sleep(2)
        except:
            break
            
    return url_empleos

# Saco la info de cada empleo:

In [21]:
def iniciar_scraper(url_empleos):
    # Inicio browser:
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()
    
    # Valores fijos:
    portal = "infoempleo"
    fecha_scrapeo = datetime.datetime.now().date()
    
    
    datos_ofertas_empleo = []
    contador = 0
    for empleo in url_empleos:
    
        browser.get(empleo)
        sleep(2)

        try:
            # Acepto cookies
            browser.find_element(By.ID, "onetrust-accept-btn-handler").click()
        except:
            pass

        # Cierro popup si aparece:
        try:
            browser.find_element(By.ID, "lightbox").find_element(By.CLASS_NAME, "close").click()
            sleep(1)
        except:
            pass

        soup_oferta = BeautifulSoup(browser.page_source, "html.parser")

        try:
            titulo = soup_oferta.find("div", class_= "title-wrapper").find("h1").text

        except:
            titulo = np.nan

        try:
            empresa = soup_oferta.find("div", class_= "title-wrapper").find("li", class_= "companyname").text

        except:
            empresa = np.nan

        try:
            presencialidad = soup_oferta.find("div", class_= "title-wrapper").find("li", class_= "badge").text

        except:
            presencialidad = np.nan    

        try:
            fecha = soup_oferta.find("div", class_= "title-wrapper").find("li", class_= "mt10").text.strip()

        except:
            fecha = np.nan

        try:
            ubicacion = soup_oferta.find("div", class_= "title-wrapper").find("li", class_= "block").text.strip()

        except:
            ubicacion = np.nan

        # Bloque de características:

        try:
            browser.find_element(By.CLASS_NAME, "areapos-vmore").click()
        except:
            pass

        bullet_points = soup_oferta.find("div", class_= "offer-excerpt").find_all("ul", class_= "inline")

        try:
            experiencia = bullet_points[0].find_all("p")[0].text
        except:
            experiencia = np.nan

        try:
            salario = bullet_points[0].find_all("p")[1].text
        except:
            salario = np.nan

        try:
            funciones = [funcion.text for funcion in bullet_points[1].find_all("li")[1:]]
        except:
            funciones = np.nan

        try:
            solicitudes = bullet_points[2].find_all("p")[1].text
        except:
            solicitudes = np.nan        

        try:
            tipo_contrato = bullet_points[3].find_all("p")[0].text
        except:
            tipo_contrato = np.nan

        try:
            jornada = bullet_points[3].find_all("p")[1].text
        except:
            jornada = np.nan  

        cuerpo_oferta = soup_oferta.find("div", class_= "offer").find_all("pre")

        try:
            descripcion = cuerpo_oferta[0].text.replace("\n", "").replace("*", "").strip()
        except:
            descripcion = np.nan    

        try:
            herramientas = cuerpo_oferta[1].text.replace("\n", "").replace("*", "").strip()
        except:
            herramientas = np.nan

        datos_ofertas_empleo.append([titulo, empresa, fecha, herramientas, descripcion, ubicacion, presencialidad, funciones, jornada, experiencia, tipo_contrato, salario, solicitudes, fecha_scrapeo, empleo, portal])
        contador += 1

        if contador % 100 == 0:
            sleep(20)
            
            
    df = pd.DataFrame(datos_ofertas_empleo, columns= ["titulo", "empresa", "fecha", "herramientas", "descripcion", "ubicacion", "presencialidad", "funciones", "jornada", "experiencia", "tipo_contrato", "salario", "solicitudes", "fecha_scrapeo", "url", "portal"])
    
    return df 

# Extraigo la url de cada empleo en la plataforma Infoempleo:

In [19]:
url_empleos = url_scraper(url_infoempleo)

# Extraigo la información de cada oferta de empleo:

In [26]:
df = iniciar_scraper(url_empleos)

# Lo guardo en la ruta de datos:
fecha = datetime.datetime.now().strftime("%d-%m-%y")
ruta_datos = "C:\\Users\\regue\\Desktop\\Data Science Projects\\PROJECTS\\IT_Job_Spain_Project\\Datos\\datos_sin_procesar\\"
nombre_archivo = f"datos_infoempleo_{fecha}.csv"
ruta_completa = os.path.join(ruta_datos, nombre_archivo)

df.to_csv(ruta_completa, index=False, sep=",")